# Блок №8

# Задание 1
___
Для чего используют оконные функции?

+ когда необходимо использовать результирующую таблицу одного запроса, как входную таблицу для другого
+ когда необходимо сохранить результирующую таблицу в новом окне
+ когда необходимо объединить две или несколько таблиц по общему ключу
+ когда необходимо получить значения соседних ячеек, не меняя при этом группировку строк

Ответ:
когда необходимо получить значения соседних ячеек, не меняя при этом группировку строк

# Задача 2
___
Соотнесите параметр с его описанием:

Ответ:
+ PARTITION BY – группирует данные для окна
+ ORDER BY – сортирует результаты внутри раздела
+ ROWS/RANGE – задает параметры рамки окна

# Задача 3
___
В чем заключается различие между ROWS и RANGE?

Ответ:
+ При использовании ROWS начало и конец рамки задаются в строках относительно текущей строки
+ При использовании RANGE начало и конец рамки задаются в разнице значений столбца из ORDER BY.

# Задача 4
___
Немного освежим в памяти порядок указания операторов в запросе! Расположите их в правильном порядке:

Ответ: \
SELECT [перечисление полей] \
FROM [имя таблицы] \
WHERE [имя поля] (=, >, <, IN,) [значение поля] \
GROUP BY [перечисление поле, по которым происходит группировка] \
HAVING [имя агрегированного поля] (=, >, <,) [значение поля] \
WINDOW [имя окна] AS [спецификация окна] \
ORDER BY [поле] (DESC/ASC) \
LIMIT [количество извлекаемых строк] \

# ЗАдача 5
___
Посмотрим на пример. Предположим, есть таблица с пользователями, состоящая из 4 колонок. Для пользователей из Германии нужно посчитать накопительную сумму покупок и записать в `TotalPerUser`.

| Country | UserID | BuyDate | Rub | TotalPerUser |
|---------|--------|---------|-----|--------------|
| Germany | 1406814346437 | 27/07/19 | 4 | 4 |
| Germany | 1406814346437 |29/07/19 | 3 | 7 |

Правильный запрос:
```
SELECT Country, UserID, BuyDate, Rub, SUM(Rub) OVER w AS TotalPerUser
FROM data
WHERE Country = 'Germany'
WINDOW w AS 
	(PARTITION BY UserID 
	 ORDER BY BuyDate ASC
         ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)
ORDER BY
	 UserID DESC, 
	 BuyDate ASC
LIMIT 100
```

# Задание 6
___
Возможно вы о них забыли, но они вернулись! Данные по продажам авокадо 🥑: 

+ `date` — дата
+ `average_price` — средняя цена одного авокадо
+ `total_volume` — количество проданных авокадо
+ `plu4046` — количество проданных авокадо PLU* 4046
+ `plu4225` — количество проданных авокадо PLU 4225
+ `lu4770` — количество проданных авокадо PLU 4770
+ `total_bags` — всего упаковок
+ `small_bags` — маленькие упаковки
+ `large_bags` — большие
+ `xlarge_bags` — очень большие
+ `type` — обычный или органический
+ `year` — год
+ `region` — город или регион (TotalUS – сразу по США)

В таблице находятся данные не за каждый день, а за конец каждой недели. Для каждой даты есть несколько наблюдений, отличающихся по типу авокадо и региону продажи. 

*PLU — код товара (Product Lookup code)

Давайте посмотрим на продажи авокадо в двух городах (NewYork, LosAngeles) и узнаем, сколько авокадо типа organic было продано в целом к концу каждой недели (накопительная сумма продаж), начиная с начала периода наблюдений (04/01/15). Пример результирующей таблицы:

| region	| date | total_volume | volume |
|-----------|------|--------------|--------|
| NewYork | 04/01/15	| 17328.24 | 17328.24 | 
| NewYork | 11/01/15 | 14817.97 | 32146.21 |
| NewYork | 18/01/15 | 9204.78 | 41350.99 |

Значения внутри окна сортируйте по дате, а саму таблицу отсортируйте по убыванию региона (сначала NY, потом LA) и по возрастанию даты.

В качестве ответа укажите объем продаж в Нью Йорке на 01/03/15 (без запятых).

Правильный запрос:
```
SELECT 
    region, 
    date, 
    total_volume, 
    SUM(total_volume) OVER w AS volume
FROM 
    avocado 
WHERE 
    region IN ('NewYork', 'LosAngeles') AND type='organic'
WINDOW w AS (
    PARTITION BY region
    ORDER BY date ASC
    ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
)
ORDER BY 
    region DESC, 
    date
```
Ответ:
```
165402.96
```

# Задание 7
___
Теперь добавьте разбиение по каждому году (year). Таким образом, в конце февраля 2016 года объем составит уже не продажи за 2015 и январь-февраль 2016, а только январь-февраль 2016.

Когда объемы продаж органических авокадо в Нью-Йорке превысили объемы продаж в Лос-Анджелесе?

Для решения задачи постройте график объема продаж в двух городах, где по оси X будет лежать дата. 

Правильный запрос:
```
SELECT 
    region, 
    date, 
    total_volume, 
    SUM(total_volume) OVER w AS volume
FROM 
    avocado 
WHERE 
    region IN ('NewYork', 'LosAngeles') AND type='organic'
WINDOW w AS (
    PARTITION BY region, year
    ORDER BY date ASC
    ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
)
ORDER BY 
    region DESC, 
    date
```
Ответ:
```
В феврале 2018
```

[ссылка н аграфик](http://redash.lab.karpov.courses/embed/query/7556/visualization/15947?api_key=0fotT4mminx1whC4h1CactKTucbJ9D6lpidJejRY&)
![](https://ucarecdn.com/f45cc8bc-1b56-45c1-a07e-55dfa24fd226/)

# Задание 8
___
Посмотрим, когда объемы продаж обычных (`conventional`) авокадо резко падали по сравнению с предыдущей неделей. Возьмите данные по США в целом, посчитайте разницу между объемом продаж в неделю x (`total_volume`) и количеством проданных авокадо в течение предыдущей недели. Значения запишите в новый столбец `week_diff`.

+ `type` – тип авокадо (conventional)
+ `region` – регион (TotalUS)
+ `total_volume` – объем продаж за неделю

Пример:

| date | total_volume | region | type | week_diff | 
|------|--------------|--------|------|-----------|
| 04/01/15 | 31,324,277.73 | TotalUS | conventional |  |
|11/01/15 | 29,063,542.75 | TotalUS | conventional | -2,260,734.98 |

Визуализируйте результат, где по оси x будет расположена дата, а по оси y –разница между продажами в x 
t
​
  и x 
t−1
​
 . Выберите верные утверждения.

Правильный запрос:
```
SELECT 
    date, 
    total_volume, 
    region, 
    total_volume - LAG(total_volume, 1) over w as week_diff
FROM 
    avocado 
WHERE 
    region = 'TotalUS' AND type='conventional'
WINDOW w AS (
    ORDER BY date ASC
    ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
)
ORDER BY  date
```
Ответ:
```
Самое крупное падение объемов продаж по сравнению с предыдущей неделей произошло 12 февраля 2017 года
Самые большие падения объемов продаж авокадо происходили в феврале
```

[ссылка на график](http://redash.lab.karpov.courses/embed/query/7558/visualization/15951?api_key=ThUtTUv8yQBqc6p68NyDwZVG4Pyimcie1ms8spyV&)
![](https://ucarecdn.com/8ca774db-2a38-478b-a05b-309d2a148ec9/)

# Задание 9
___
Посмотрим более подробно на объемы продаж авокадо в Нью-Йорке (NewYork) в 2018 году. Создайте колонку с разницей объемов продаж за неделю и за неделю до этого для каждого типа авокадо. Найдите день, когда продажи авокадо типа organic увеличились по сравнению с предыдущей неделей, а conventional – наоборот упали. Если таких дней несколько, то укажите их через запятую с пробелом, формат – 31/12/2020.

Правильный запрос:
```
SELECT 
    type,
    date, 
    total_volume - LAG(total_volume, 1) over w as week_diff
FROM 
    avocado 
WHERE 
    region = 'NewYork' and year = 2018
WINDOW w AS (
    PARTITION BY type
    ORDER BY date ASC
    ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
)
ORDER BY  
    type,
    date
```
Ответ:
```
18/02/2018, 18/03/2018
```

[](http://redash.lab.karpov.courses/embed/query/7560/visualization/15955?api_key=CXDGl9ejELnvkuD6f3KnLDV7mVYDsHCp4gepnptB&)
![](https://ucarecdn.com/dc48a9f0-f0b2-4afb-be1f-db63947a31dc/)

# Задание 10
___
Теперь посчитайте скользящее среднее цены авокадо (`average_price`) в Нью-Йорке с разбивкой по типу авокадо. В качестве окна используйте текущую неделю и предыдущие две (обратите внимание, что в данной таблице в строках содержатся данные за неделю, а не за один день). Например 04/01/15, 11/01/15 и 18/01/15 для подсчета значения для 18/01/15.

| date | verage_price | region | type | rolling_price |
|------|--------------|--------|------|---------------|
| 04/01/15 | 1.09 | NewYork | conventional | 1.09 |
| 11/01/15 | 1.34 | NewYork | conventional | 1.22 |
| 18/01/15 | 1.37 | NewYork | conventional | 1.27 |
| 25/01/15 | 1.36 | NewYork | conventional | 1.36 |

В качестве ответа укажите полученное значение для обычных (conventional) авокадо за 17/04/16.

Правильный запрос:
```
SELECT 
    type,
    date, 
    average_price,
    avg(average_price) over w as rolling_price
FROM 
    avocado 
WHERE 
    region = 'NewYork'
WINDOW w AS (
    PARTITION BY type
    ORDER BY date ASC
    ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
)
ORDER BY  
    type,
    date
```
Ответ:
```
1.16
```

[ссылка на график](http://redash.lab.karpov.courses/embed/query/7562/visualization/15958?api_key=8HO8XAfKdqF0k6y1dOthixpBCSrErkLPV9cup2HH&)
![](https://ucarecdn.com/586a2dc7-331f-4022-99fb-91e4bdf5a7a5/)